In [1]:
library(tmle)

Loading required package: glmnet

Loading required package: Matrix

Loaded glmnet 4.1-1

Loading required package: SuperLearner

Loading required package: nnls

Loading required package: gam

Loading required package: splines

Loading required package: foreach

Loaded gam 1.20


Super Learner

Version: 2.0-28

Package created on 2021-05-04


Welcome to the tmle package, version 1.5.0-1

Major changes since v1.3.x. Use tmleNews() to see details on changes and bug fixes



In [2]:
set.seed(1)
n <- 250
W <- matrix(rnorm(n * 3), ncol = 3)
A <- rbinom(n, 1, 1/(1 + exp(-(0.2 * W[, 1] - 0.1 * W[, 2] + 0.4 * W[, 3]))))
Y <- A + 2 * W[, 1] + W[, 3] + W[, 2]^2 + rnorm(n)

In [3]:
result1 <- tmle(Y, A, W)
summary(result1)

Loading required namespace: dbarts



 Initial estimation of Q
	 Procedure: cv-SuperLearner, ensemble
	 Model:
		 Y ~  SL.glm_All + tmle.SL.dbarts2_All + SL.glmnet_All

	 Coefficients: 
	      SL.glm_All    0 
	 tmle.SL.dbarts2_All    1 
	   SL.glmnet_All    0 

	 Cross-validated R squared :  0.8351 

 Estimation of g (treatment mechanism)
	 Procedure: SuperLearner, ensemble	 Empirical AUC = 0.6363 

	 Model:
		 A ~  SL.glm_All + tmle.SL.dbarts.k.5_All + SL.gam_All 

	 Coefficients: 
	      SL.glm_All    0.6068355 
	 tmle.SL.dbarts.k.5_All    0 
	      SL.gam_All    0.3931645 

 Estimation of g.Z (intermediate variable assignment mechanism)
	 Procedure: No intermediate variable 

 Estimation of g.Delta (missingness mechanism)
	 Procedure: No missingness, ensemble

 Bounds on g: (0.0573, 1) 

 Bounds on g for ATT/ATE: (0.0573, 0.9427) 

 Additive Effect
   Parameter Estimate:  0.8465
   Estimated Variance:  0.028615
              p-value:  5.6094e-07
    95% Conf Interval: (0.51495, 1.1781) 

 Additive Effect among the Trea

In [4]:
n <- 250
W <- matrix(rnorm(n * 3), ncol = 3)
colnames(W) <- paste("W", 1:3, sep = "")
A <- rbinom(n, 1, plogis(0.6 * W[, 1] + 0.4 * W[, 2] + 0.5 * W[, 3]))
Y <- rbinom(n, 1, plogis(A + 0.2 * W[, 1] + 0.1 * W[, 2] + 0.2 * W[, 3]^2))

In [5]:
result2 <- tmle(Y, A, W, family = "binomial", Qform = Y ~ A + W1 + W2 + W3, gform = A ~
    W1 + W2 + W3)
summary(result2)

 Initial estimation of Q
	 Procedure: cv-glm, user-supplied model
	 Cross-validated pseudo R squared :  0.0417 

 Estimation of g (treatment mechanism)
	 Procedure: user-supplied regression formula, ensemble	 Empirical AUC = 0.7515 

	 Model:
		 A ~  (Intercept) + W1 + W2 + W3 

	 Coefficients: 
	     (Intercept)   -0.1125226 
	              W1    0.6760319 
	              W2    0.4934346 
	              W3    0.6834868 

 Estimation of g.Z (intermediate variable assignment mechanism)
	 Procedure: No intermediate variable 

 Estimation of g.Delta (missingness mechanism)
	 Procedure: No missingness, ensemble

 Bounds on g: (0.0573, 1) 

 Bounds on g for ATT/ATE: (0.0573, 0.9427) 

 Additive Effect
   Parameter Estimate:  0.2767
   Estimated Variance:  0.0051925
              p-value:  0.0001231
    95% Conf Interval: (0.13546, 0.41793) 

 Additive Effect among the Treated
   Parameter Estimate:  0.26757
   Estimated Variance:  0.006075
              p-value:  0.00059723
    95% Conf Int

In [6]:
Y <- W[, 1] + W[, 2]^2 + rnorm(n)
Delta <- rbinom(n, 1, 1/(1 + exp(-(1.7 - 1 * W[, 1]))))

In [7]:
# result3 <- tmle(Y, A = NULL, W, Delta = Delta, Qform = "Y~A+W1+W2+W3")
# print(result3)

In [8]:
A <- rbinom(n, 1, 0.5)
Z <- rbinom(n, 1, plogis(0.5 * A + 0.1 * W[, 1]))
Y <- 1 + A + 10 * Z + W[, 1] + rnorm(n)

In [9]:
CDE <- tmle(Y, A, W, Z, gform = "A~1", g.Zform = "Z ~ A + W1")
print(CDE)
total.effect <- tmle(Y, A, W, gform = "A~1")
print(total.effect)

Controlled Direct Effect
           ----- Z = 0 -----
 Additive Effect
   Parameter Estimate:  1.2868
   Estimated Variance:  0.034427
              p-value:  4.0558e-12
    95% Conf Interval: (0.92313, 1.6505) 

           ----- Z = 1 -----
 Additive Effect
   Parameter Estimate:  0.85656
   Estimated Variance:  0.036668
              p-value:  7.7083e-06
    95% Conf Interval: (0.48124, 1.2319) 
 Additive Effect
   Parameter Estimate:  1.9721
   Estimated Variance:  0.38986
              p-value:  0.0015858
    95% Conf Interval: (0.74833, 3.1959) 

 Additive Effect among the Treated
   Parameter Estimate:  2.0087
   Estimated Variance:  0.38971
              p-value:  0.0012926
    95% Conf Interval: (0.78509, 3.2322) 

 Additive Effect among the Controls
   Parameter Estimate:  1.9363
   Estimated Variance:  0.39127
              p-value:  0.0019644
    95% Conf Interval: (0.71031, 3.1623) 
